In [35]:
link_UNITY_select_all_active_ideas = 'https://rest.unity.finance/api/v1/tradeIdeas?limit=1000&statuses=ACTIVE'
UNITY_KEY = 'c3f28d5b-9694-4307-b30f-6f800ee13506'
import requests
import pandas as pd

In [36]:
def Unity_api(link:str) -> list:
    headers = {'auth-token' : UNITY_KEY}
    response = requests.get(link, headers=headers)
    return response.json()

In [37]:
df = pd.DataFrame(Unity_api(link_UNITY_select_all_active_ideas)['items'])

In [38]:
df['expirationTime'] = pd.to_datetime(df['expirationTime'], format='mixed')
df['expirationTime'] = df['expirationTime'].dt.strftime("%Y-%m-%d %H:%M:%S")
df['createTime'] = pd.to_datetime(df['createTime'], format='mixed')
df['createTime'] = df['createTime'].dt.strftime("%Y-%m-%d %H:%M:%S")

df['instrumentIds'] = df['instrumentIds'].apply(lambda x: x[0])
df

,id,brokerId,instrumentIds,openOrderType,side,price,stopLoss,takeProfit,confidence,expirationTime,...,status,publishTime,createTime,updateTime,source,provider,state,subscribers,priority,rate
0,7065,1,16700,LIMIT,BUY,3.3500,3.1300,3.8500,40.0,2024-08-26 20:00:00,...,ACTIVE,None,2024-07-26 13:20:26,2024-07-26T13:23:47Z,CONNECTOR,SIGNAL_CENTRE,None,0,NaN,NaN
1,7373,1,14333,LIMIT,BUY,77.8000,76.8600,79.6800,40.0,2024-08-09 20:00:00,...,ACTIVE,None,2024-08-08 12:01:08,2024-08-08T12:24:14Z,CONNECTOR,SIGNAL_CENTRE,None,0,NaN,NaN
2,7380,1,20228,LIMIT,BUY,78.5900,77.5900,81.1300,60.0,2024-08-09 20:00:00,...,ACTIVE,None,2024-08-09 05:37:59,2024-08-09T05:40:18Z,CONNECTOR,SIGNAL_CENTRE,None,0,NaN,NaN
3,7381,1,41,LIMIT,SELL,149.5000,151.0000,145.2200,60.0,2024-08-09 20:00:00,...,ACTIVE,None,2024-08-09 05:41:13,2024-08-09T05:41:32Z,CONNECTOR,SIGNAL_CENTRE,None,0,NaN,NaN
4,7382,1,47,LIMIT,SELL,1.0954,1.0984,1.0882,40.0,2024-08-09 20:00:00,...,ACTIVE,None,2024-08-09 05:41:57,2024-08-09T05:44:57Z,CONNECTOR,SIGNAL_CENTRE,None,0,NaN,NaN
5,7383,1,30,LIMIT,BUY,0.6550,0.6535,0.6610,60.0,2024-08-09 20:00:00,...,ACTIVE,None,2024-08-09 05:40:23,2024-08-09T05:45:52Z,CONNECTOR,SIGNAL_CENTRE,None,0,NaN,NaN
6,7384,1,59518,LIMIT,BUY,2.0600,2.0100,2.2000,40.0,2024-08-09 20:00:00,...,ACTIVE,None,2024-08-09 05:45:26,2024-08-09T05:48:18Z,CONNECTOR,SIGNAL_CENTRE,None,0,NaN,NaN
7,7385,1,26,LIMIT,BUY,0.9375,0.9335,0.9535,60.0,2024-08-09 20:00:00,...,ACTIVE,None,2024-08-09 05:45:25,2024-08-09T05:49:14Z,CONNECTOR,SIGNAL_CENTRE,None,0,NaN,NaN
8,7386,1,37,LIMIT,SELL,1.2805,1.2840,1.2700,60.0,2024-08-09 20:00:00,...,ACTIVE,None,2024-08-09 05:55:08,2024-08-09T05:52:56Z,CONNECTOR,SIGNAL_CENTRE,None,0,NaN,NaN
9,7387,1,48360,LIMIT,BUY,38785.0000,38435.0000,39650.0000,80.0,2024-08-09 20:00:00,...,ACTIVE,None,2024-08-09 05:56:07,2024-08-09T05:59:13Z,CONNECTOR,SIGNAL_CENTRE,None,0,NaN,NaN


In [39]:
instrumentIds = df['instrumentIds'].tolist()
instrumentIds = ",".join(map(str, instrumentIds))

link = f'https://rest.unity.finance/api/v1/instrumentDetails?instrumentId={instrumentIds}'
data = Unity_api(link)
ticker_df = pd.DataFrame(data['items'])
ticker_df = ticker_df[['id','code']]
df = pd.merge(df, ticker_df, left_on='instrumentIds', right_on='id', how='left')
df["instrumentIds"] = df['code']
#============================

df = df.drop(['brokerId','status', 'publishTime','updateTime','id_y','code'], axis=1)
df.columns = [['unity_id','instrument','open_order_type','side', 'open_order_price','stop_loss', 'take_profit','confidence','expiration_time','description_text','create_time', "source", "provider", "state", "subscribers",	"priority", "rate"]]
#============================

In [40]:
df[['instrument', 'side', 'open_order_price', 'stop_loss',	'take_profit',	"expiration_time",	"create_time",	'source', 'description_text']]

,instrument,side,open_order_price,stop_loss,take_profit,expiration_time,create_time,source,description_text
0,PTON.NASDAQ,BUY,3.3500,3.1300,3.8500,2024-08-26 20:00:00,2024-07-26 13:20:26,CONNECTOR,"[{'locale': 'RU', 'target': 'Мы намереваемся п..."
1,BABA.NYSE,BUY,77.8000,76.8600,79.6800,2024-08-09 20:00:00,2024-08-08 12:01:08,CONNECTOR,"[{'locale': 'RU', 'target': 'Мы намереваемся п..."
2,UKOil,BUY,78.5900,77.5900,81.1300,2024-08-09 20:00:00,2024-08-09 05:37:59,CONNECTOR,"[{'locale': 'RU', 'target': 'Мы намереваемся п..."
3,USDJPY,SELL,149.5000,151.0000,145.2200,2024-08-09 20:00:00,2024-08-09 05:41:13,CONNECTOR,"[{'locale': 'RU', 'target': 'Мы намереваемся п..."
4,EURUSD,SELL,1.0954,1.0984,1.0882,2024-08-09 20:00:00,2024-08-09 05:41:57,CONNECTOR,"[{'locale': 'RU', 'target': 'Мы намереваемся п..."
5,AUDUSD,BUY,0.6550,0.6535,0.6610,2024-08-09 20:00:00,2024-08-09 05:40:23,CONNECTOR,"[{'locale': 'RU', 'target': 'Мы намереваемся п..."
6,NG,BUY,2.0600,2.0100,2.2000,2024-08-09 20:00:00,2024-08-09 05:45:26,CONNECTOR,"[{'locale': 'RU', 'target': 'Мы намереваемся п..."
7,EURCHF,BUY,0.9375,0.9335,0.9535,2024-08-09 20:00:00,2024-08-09 05:45:25,CONNECTOR,"[{'locale': 'RU', 'target': 'Мы намереваемся п..."
8,GBPUSD,SELL,1.2805,1.2840,1.2700,2024-08-09 20:00:00,2024-08-09 05:55:08,CONNECTOR,"[{'locale': 'RU', 'target': 'Мы намереваемся п..."
9,US30,BUY,38785.0000,38435.0000,39650.0000,2024-08-09 20:00:00,2024-08-09 05:56:07,CONNECTOR,"[{'locale': 'RU', 'target': 'Мы намереваемся п..."
